In [1]:
import numpy as np # 快速操作结构数组的工具
import matplotlib.pyplot as plt  # 可视化绘制
from sklearn.linear_model import LinearRegression  # 线性回归
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.preprocessing import OneHotEncoder


# 弱分类器的数目
n_estimator = 10
# 随机生成分类数据。
X, y = make_classification(n_samples=80000,n_features=20,n_classes=2)

# 切分为测试集和训练集，比例0.5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
# 将训练集切分为两部分，一部分用于训练GBDT模型，另一部分输入到训练好的GBDT模型生成GBDT特征，然后作为LR的特征。这样分成两部分是为了防止过拟合。
X_train_gbdt, X_train_lr, y_train_gbdt, y_train_lr = train_test_split(X_train, y_train, test_size=0.5)
# 调用GBDT分类模型。
gbdt = GradientBoostingClassifier(n_estimators=n_estimator)
# 调用one-hot编码。
one_hot = OneHotEncoder()
# 调用LR分类模型。
lr = LogisticRegression()


'''使用X_train训练GBDT模型，后面用此模型构造特征'''
gbdt.fit(X_train_gbdt, y_train_gbdt)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [2]:
X_leaf_index = gbdt.apply(X_train_gbdt)

In [5]:
np.shape(X_leaf_index)

(20000, 10, 1)

In [6]:
X_leaf_index = gbdt.apply(X_train_gbdt)[:, :, 0]

In [7]:
np.shape(X_leaf_index)

(20000, 10)

In [9]:
X_lr_leaf_index = gbdt.apply(X_train_lr)[:, :, 0] # apply返回每个样本在每科树中所属的叶子节点索引。行数为样本数，列数为树数目。值为在每个数的叶子索引
print('每个样本在每个树中所属的叶子索引\n',X_leaf_index)


每个样本在每个树中所属的叶子索引
 [[11. 11. 11. ... 11. 14. 11.]
 [ 3.  3.  3. ...  4.  3.  3.]
 [ 3.  3.  3. ...  4.  3.  3.]
 ...
 [11. 11. 11. ... 11. 11. 11.]
 [11. 11. 10. ... 11. 10. 11.]
 [11. 11. 11. ... 11. 14. 11.]]


In [10]:
# fit one-hot编码器
one_hot.fit(X_leaf_index)  # 训练one-hot编码，就是识别每列有多少可取值
X_lr_one_hot = one_hot.transform(X_lr_leaf_index)  # 将训练数据，通过gbdt树，形成的叶子节点（每个叶子代表了原始特征的一种组合）索引，编码成one0-hot特征。
# 编码后的每个特征代表原来的一批特征的组合。


In [11]:
''' 
使用训练好的GBDT模型构建特征，然后将特征经过one-hot编码作为新的特征输入到LR模型训练。
'''

# 使用lr训练gbdt的特征组合
print('使用逻辑回归训练GBDT组合特征的结果')
lr.fit(X_lr_one_hot, y_train_lr)

使用逻辑回归训练GBDT组合特征的结果


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
# 用训练好的LR模型多X_test做预测
y_pred_grd_lm = lr.predict_proba(one_hot.transform(gbdt.apply(X_test)[:, :, 0]))[:, 1]  # 获取测试集正样本的概率


In [13]:

y_pre= lr.predict(one_hot.transform(gbdt.apply(X_test)[:, :, 0]))  # 获取测试集正样本的概率
 
y_pro= lr.predict_proba(one_hot.transform(gbdt.apply(X_test)[:, :, 0]))[:, 1]  # 获取测试集正样本的概率



In [15]:
from sklearn import metrics


print("AUC Score :",(metrics.roc_auc_score(y_test, y_pro)))   
print("Accuracy :",(metrics.accuracy_score(y_test, y_pre)))  

AUC Score : 0.992947907298495
Accuracy : 0.9827
